### Titanic survived passenger prediction.
#### It's a Kaggle competation dataset and we do not have actual labels with us. we need to submit our prediction in the kaggle competation.
#### This notebook has preprocessing steps and applying multiple classification algos and submitting top score model prediction.
#### We need to apply our general analogy about data set features and move forward.
#### You will be able to download datasets in kaggle

In [1]:
# Import necessory libraries
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Explore
df_train = pd.read_csv('train.csv',sep=',')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train.shape

(891, 12)

In [4]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
def titanic(df):
# STEP1  After analysing Columns such as 'PassengerId','Cabin','Ticket','Name','Fare','Embarked' do not add values, so we are dropping.    
    df.drop(columns=['PassengerId','Cabin','Ticket','Name','Fare','Embarked'],axis=1,inplace=True)

# STEP 2   Converting Categorical into Numerical      
    df['Sex'].replace('male', 1,inplace=True)
    df['Sex'].replace('female', 0,inplace=True)
    
# STEP 3  Since Age is important factor and taking average age in each class w.r.t Age for furthure imputation    
    class_one_male = df.Age[(df['Pclass'] == 1) & (df['Sex'] == 1 )].mean()
    class_one_female = df.Age[(df['Pclass'] == 1) & (df['Sex'] == 0)].mean()
    class_two_male = df.Age[(df['Pclass'] == 2) & (df['Sex'] == 1)].mean()
    class_two_female = df.Age[(df['Pclass'] == 2) & (df['Sex'] == 0)].mean()
    class_three_male = df.Age[(df['Pclass'] == 3) & (df['Sex'] == 1)].mean()
    class_three_female = df.Age[(df['Pclass'] == 3) & (df['Sex'] == 0)].mean()

    
# STEP 4  We are seperating passenger of each class w.r.t gender so that we can impute missing age accordingly     
    df_male_o = df[(df['Pclass'] == 1) & (df['Sex'] == 1)]
    df_female_o= df[(df['Pclass'] == 1) & (df['Sex'] == 0)]
    df_male_t = df[(df['Pclass'] == 2) & (df['Sex'] == 1)]
    df_female_t= df[(df['Pclass'] == 2) & (df['Sex'] == 0)]
    df_male_th = df[(df['Pclass'] == 3) & (df['Sex'] == 1)]
    df_female_th= df[(df['Pclass'] == 3) & (df['Sex'] == 0)]

# STEP 5 Null Imputation for Age
    df_male_o['Age'].fillna(value=class_one_male,inplace=True)
    df_female_o['Age'].fillna(value=class_one_female,inplace=True)
    df_male_t['Age'].fillna(value=class_two_male,inplace=True)
    df_female_t['Age'].fillna(value=class_two_female,inplace=True)
    df_male_th['Age'].fillna(value=class_three_male,inplace=True)
    df_female_th['Age'].fillna(value=class_three_female,inplace=True)

# STEP 6 Cancating above datasets   
    dataframes = [df_male_o,df_female_o,df_male_t,df_female_t,df_male_th,df_female_th]
    df_structured = pd.concat(dataframes)
    return df_structured
    

In [12]:
# Calling function
df = titanic(df = pd.read_csv('titanic_train.csv',sep=','))

# Check if any null exists
print (df.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
dtype: int64


In [17]:
# Check for any data loss
print ('Final Shape = >',df.shape)

# training data sets
X_train,y_train = df.drop('Survived',axis=1),df.Survived


Final Shape = > (891, 6)


In [8]:
# scoring metrics
from sklearn.metrics import scorer

In [14]:
# Logistic model for training
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
# Importing test set for prediction
x_test= titanic(pd.read_csv('titanic_test.csv',sep=','))
log_pred = log.predict(x_test)
log_score = round(log.score(X_train,y_train)* 100,2)
print ('Logistic training Score ==>', log_score)

Logistic training Score ==> 81.59


In [22]:
# Knn Classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [61]:
knn_pred = knn.predict(x_test)
knn_score = round(knn.score(X_train,y_train)* 100,2)
print ('Knn training Score ==>', knn_score)

Knn training Score ==> 85.41


In [60]:
# Support Vector Machine
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(x_test)
svc_score = round(svc.score(X_train, y_train) * 100, 2)
print ('SVC training Score ==>',svc_score)

SVC training Score ==> 64.2


In [56]:
# Sthocastic Gradiant Descent Classifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(x_test)
sgd_score = round(sgd.score(X_train, y_train) * 100, 2)
print ('SGDC training Score ==>',sgd_score)

SGDC training Score ==> 71.04


In [58]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy')
decision_tree.fit(X_train, y_train)
dt_pred = decision_tree.predict(x_test)
dt_score = round(decision_tree.score(X_train, y_train) * 100, 2)
print ('Dic Tree training Score ==>',dt_score)

Dic Tree training Score ==> 92.03


In [59]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=50,criterion='entropy')
random_forest.fit(X_train, y_train)
rm_pred = random_forest.predict(x_test)
random_forest.score(X_train, y_train)
rm_score = round(random_forest.score(X_train, y_train) * 100, 2)
print ('Random Forest training Score ==>',rm_score)

Random Forest training Score ==> 92.03


In [64]:
models = pd.DataFrame({'Classifier':['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Stochastic Gradient Decent', 'Decision Tree'],
                      'Score':[svc_score,knn_score,log_score,rm_score,sgd_score,dt_score]})
models.sort_values(by='Score',ascending=False)

,Classifier,Score
3,Random Forest,92.03
5,Decision Tree,92.03
1,KNN,85.41
2,Logistic Regression,81.59
4,Stochastic Gradient Decent,71.04
0,Support Vector Machines,64.20


In [74]:
# Random forest seems to be a best classifier, so will be submitting the same results.
titanic_test = pd.read_csv('titanic_test.csv',sep=',')
submission = pd.DataFrame({'PassengerId': titanic_test['PassengerId'],
                          'Survived': dt_pred})

In [75]:
# Final submission
submission.to_csv('titanic_submission.csv',index=False)